In [1]:
!ls

Untitled.ipynb


In [2]:
!ls

Data_understanding.ipynb
Untitled.ipynb


DATA UNDERSTANDING

In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
pd.set_option('display.max_rows',500)

In [5]:
data_path='../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd_raw = pd.read_csv(data_path)

In [6]:
pd_raw.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/9/22,6/10/22,6/11/22,6/12/22,6/13/22,6/14/22,6/15/22,6/16/22,6/17/22,6/18/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,180864,180864,180864,180864,181120,181178,181236,181465,181534,181574
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,276583,276638,276690,276731,276731,276821,276821,276821,277141,277141
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,265920,265925,265925,265927,265937,265943,265952,265964,265968,265971
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,43224,43224,43224,43224,43224,43224,43449,43449,43449,43449
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,99761,99761,99761,99761,99761,99761,99761,99761,99761,99761


Webscrapping

In [7]:
page = requests.get("https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html")

In [8]:
soup = BeautifulSoup(page.content, 'html.parser')

In [10]:
html_table=soup.find('table')

In [17]:
all_rows=html_table.find_all('tr')
final_data_list=[]

In [19]:
for pos,rows in enumerate(all_rows):
    #print(pos)
    #print(rows)
    col_list=[each_col.get_text(strip=True) for each_col in rows.find_all('td')]
    final_data_list.append(col_list)
    
    #for each_col in rows.find_all('td'):
        #print(each_col.get_text(strip=True))

In [25]:
pd_daily_status=pd.DataFrame(final_data_list).dropna().rename(columns={0:'state',
                                                       1:'cases',
                                                       2:'difference to the previous day',
                                                       3:'cases in the last 7 days',
                                                       4:'7-day incidence',
                                                       5:'deaths'})

In [26]:
pd_daily_status.head()

,state,cases,difference to the previous day,cases in the last 7 days,7-day incidence,deaths
2,Baden-Württem­berg,3.747.115,0,30.160,"271,6",16.219
3,Bayern,5.007.148,0,47.415,"360,8",24.220
4,Berlin,1.072.567,2.033,10.573,"288,6",4.635
5,Branden­burg,804.553,1.120,7.105,"280,7",5.699
6,Bremen,207.926,595,3.589,"527,7",782


API calls

In [36]:
data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronaf%C3%A4lle_in_den_Bundesl%C3%A4ndern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

In [37]:

json_object=json.loads(data.content)
type(json_object)

dict

In [38]:
json_object.keys()

dict_keys(['objectIdFieldName', 'uniqueIdField', 'globalIdFieldName', 'geometryProperties', 'geometryType', 'spatialReference', 'fields', 'features'])

In [40]:
full_list =[]
for pos,each_dict in enumerate (json_object['features'][:]):
    full_list.append(each_dict['attributes'])

In [42]:
pd_full_list = pd.DataFrame(full_list)
pd_full_list.head()

,OBJECTID_1,LAN_ew_AGS,LAN_ew_GEN,LAN_ew_BEZ,LAN_ew_EWZ,OBJECTID,Fallzahl,Aktualisierung,AGS_TXT,GlobalID,faelle_100000_EW,Shape__Area,Shape__Length,Death,cases7_bl_per_100k,cases7_bl,death7_bl,cases7_bl_per_100k_txt,AdmUnitId
0,1,01,Schleswig-Holstein,Land,2910875,15,786715,1655589600000,01,fc5ba936-c95c-432c-8a33-9eb2f30b660f,27026.753124,4.573731e+10,2.881496e+06,2586,706.282475,20559,0,"706,3",1
1,2,02,Hamburg,Freie und Hansestadt,1852478,6,608227,1655589600000,02,0f3e860c-5181-4d3f-a421-1d51f50315ea,32833.156453,2.089396e+09,4.188002e+05,2695,465.052756,8615,2,"465,1",2
2,3,03,Niedersachsen,Land,8003421,9,2496938,1655589600000,03,3fd77024-c29b-4843-9be8-682ad48e60c9,31198.383791,1.299836e+11,4.008988e+06,9492,674.948875,54019,2,"674,9",3
3,4,04,Bremen,Freie Hansestadt,680130,5,208467,1655589600000,04,4132268b-54de-4327-ac1e-760e915112f1,30651.052005,1.119157e+09,3.357177e+05,782,499.022246,3394,0,"499,0",4
4,5,05,Nordrhein-Westfalen,Land,17925570,10,5495937,1655589600000,05,561d658f-3ee5-46e3-bc95-3528c6558ab9,30659.761447,8.782936e+10,2.648673e+06,25588,462.166615,82846,5,"462,2",5


In [44]:
pd_full_list.to_csv('../data/raw/NPGEO/GER_state_data.csv',sep=';')

API US data